# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [ ]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

In [ ]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

In [ ]:
def load_pose_data(i):
    '''load pose data from file'''
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = np.array(pickle.load(open(filename, 'rb')))
    target = np.array([i] * data.shape[0])
    return data, target

In [ ]:
# load all the data

dt_tuples = [load_pose_data(i) for i, _ in enumerate(classes)]

all_data = np.vstack([x[0] for x in dt_tuples])
all_target = np.concatenate([x[1] for x in dt_tuples])

N = all_data.shape[0]

print('total number of data', len(all_data))

In [ ]:
# shuffule data
permutation = np.random.permutation(N)
n_training_data = int(N * 0.7)
training_data = permutation[:n_training_data]
test_data = permutation[n_training_data:]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [ ]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [ ]:
clf.fit(all_data[training_data,:], all_target[training_data])

### predicting

In [ ]:
clf.predict(all_data[-1, None]), all_target[-1]

In [ ]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [ ]:
expected = all_target[training_data]
predicted = clf.predict(all_data[training_data])

evaluate(expected, predicted)

## 4. Evaluate on the test data

In [ ]:
predicted = clf.predict(all_data[test_data,:])
expected = all_target[test_data]

evaluate(expected, predicted)

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [ ]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [ ]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1,None]), all_target[-1]